In [1]:
import os
os.chdir("..")

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from pathlib import Path
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from typing import Dict

from engine.attention import AttentionAudioClassifier
from engine.data import get_data_loader, get_label_mapping, get_main_classes_loader, get_commands_loader

plt.xticks(rotation=90)
plt.clf()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def get_labels_and_predictions(model, loader):
    all_labels, all_predictions = [], []
    for X, y in loader:
        prediction = model(X).argmax(dim=1)
        all_labels.append(y.cpu().numpy())
        all_predictions.append(prediction.cpu().numpy())
    labels = np.concatenate(all_labels)
    predictions = np.concatenate(all_predictions)
    return labels, predictions

In [ ]:
def get_composed_model_labels_and_predictions(main_model, command_model, loader):
    all_labels, all_predictions = [], []
    for X, y in loader:
        prediction = main_model(X).argmax(dim=1)
        if (prediction == COMMAND_LABEL).sum() > 0:
            prediction[prediction == COMMAND_LABEL] = commands_model(X[prediction == COMMAND_LABEL]).argmax(dim=1)
        all_labels.append(y.cpu().numpy())
        all_predictions.append(prediction.cpu().numpy())
    labels = np.concatenate(all_labels)
    predictions = np.concatenate(all_predictions)
    return labels, predictions

## All labels

In [ ]:
## Load
models_paths = list(Path("results/attention_all_classes").rglob("*.ckpt"))
loader = get_data_loader("test")

In [ ]:
## Get accuracies
accuracies_all_labels = []
for path in models_paths:
    model = AttentionAudioClassifier.load_from_checkpoint(path)
    labels, predictions = get_labels_and_predictions(model, loader)
    accuracies_all_labels.append((labels == predictions).mean())

In [ ]:
# Confusion matrix
best_model_path = models_paths[np.argmax(accuracies_all_labels)]
best_model = AttentionAudioClassifier.load_from_checkpoint(best_model_path)
labels, predictions = get_labels_and_predictions(model, loader)

mapping = get_label_mapping("all")
matrix = confusion_matrix(labels, predictions)
matrix = (matrix.T / matrix.sum(axis=1)).T
display_labels=[mapping[label] for label in sorted(np.unique(labels))]

fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(matrix, ax=ax, annot=True, cmap=sns.light_palette("seagreen", as_cmap=True))
ax.set_xticklabels(display_labels, rotation=90)
ax.set_title("Confusion matrix of attention model on all labels")

## Main classes

In [ ]:
## Load
models_paths = list(Path("results/attention_main_classes").rglob("*.ckpt"))
loader = get_main_classes_loader("test")

In [ ]:
## Get accuracies
accuracies_main_labels = []
for path in models_paths:
    model = AttentionAudioClassifier.load_from_checkpoint(path)
    labels, predictions = get_labels_and_predictions(model, loader)
    accuracies_main_labels.append((labels == predictions).mean())

In [ ]:
# Confusion matrix
best_model_path = models_paths[np.argmax(accuracies_main_labels)]
best_model = AttentionAudioClassifier.load_from_checkpoint(best_model_path)
labels, predictions = get_labels_and_predictions(model, loader)

mapping = get_label_mapping("main")
matrix = confusion_matrix(labels, predictions)
matrix = (matrix.T / matrix.sum(axis=1)).T
display_labels=[mapping[label] for label in sorted(np.unique(labels))]

fig, ax = plt.subplots(figsize=(4, 4))
sns.heatmap(matrix, ax=ax, annot=True, cmap=sns.light_palette("seagreen", as_cmap=True))
ax.set_xticklabels(display_labels, rotation=90)
ax.set_title("Confusion matrix of attention model on main labels")

## Commands only

In [ ]:
## Load
models_paths = list(Path("results/attention_commands_only").rglob("*.ckpt"))
loader = get_commands_loader("test")

In [ ]:
## Get accuracies
accuracies_commands_labels = []
for path in models_paths:
    model = AttentionAudioClassifier.load_from_checkpoint(path)
    labels, predictions = get_labels_and_predictions(model, loader)
    accuracies_commands_labels.append((labels == predictions).mean())

In [ ]:
# Confusion matrix
best_model_path = models_paths[np.argmax(accuracies_commands_labels)]
best_model = AttentionAudioClassifier.load_from_checkpoint(best_model_path)
labels, predictions = get_labels_and_predictions(model, loader)

mapping = get_label_mapping("commands_only")
matrix = confusion_matrix(labels, predictions)
matrix = (matrix.T / matrix.sum(axis=1)).T
display_labels=[mapping[label] for label in sorted(np.unique(labels))]

fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(matrix, ax=ax, annot=True, cmap=sns.light_palette("seagreen", as_cmap=True))
ax.set_xticklabels(display_labels, rotation=90)
ax.set_title("Confusion matrix of attention model on only commands")

## Composed model

In [5]:
import pickle as pkl
with open("../data/converted/encoder.pkl", "rb") as f:
    encoder = pkl.load(f)

ModuleNotFoundError: No module named 'sklearn'

In [1]:
COMMAND_LABEL = 0

In [ ]:
## Load
main_models_paths = list(Path("results/attention_main_classes").rglob("*.ckpt"))
commands_models_paths = list(Path("results/attention_commands_only").rglob("*.ckpt"))
loader = get_data_loader("test")

In [ ]:
accuracies_composed_model = []

for main_model_path, commands_model_path in zip(main_models_paths, commands_models_path):
    main_model = AttentionAudioClassifier.load_from_checkpoint(main_model_path)
    commands_model = AttentionAudioClassifier.load_from_checkpoint(commands_model_path)
    accuracies_composed_model.append(
        get_composed_model_labels_and_predictions(main_model, composed_model, loader)
    )

In [ ]:
# Confusion matrix
best_model_idx = np.argmax(accuracies_composed_model)
best_main_model = AttentionAudioClassifier.load_from_checkpoint(main_models_paths[best_model_idx])
best_commands_model = AttentionAudioClassifier.load_from_checkpoint(commands_models_paths[best_model_idx])
labels, predictions = get_composed_model_labels_and_predictions(best_main_model, best_commands_model, loader)

mapping = get_label_mapping("all")
matrix = confusion_matrix(labels, predictions)
matrix = (matrix.T / matrix.sum(axis=1)).T
display_labels=[mapping[label] for label in sorted(np.unique(labels))]

fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(matrix, ax=ax, annot=True, cmap=sns.light_palette("seagreen", as_cmap=True))
ax.set_xticklabels(display_labels, rotation=90)
ax.set_title("Confusion matrix of composed attention model")